In [1]:
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

In [2]:
sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [8]:
estimator = TensorFlow(entry_point='model.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.m5.xlarge',
                             framework_version='2.1.0',
                             py_version='py3',
                             distributions={'parameter_server': {'enabled': True}})

Parameter distribution will be renamed to {'parameter_server': {'enabled': True}} in SageMaker Python SDK v2.


In [9]:
estimator.fit()

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-06-30 06:42:31 Starting - Starting the training job...
2020-06-30 06:42:33 Starting - Launching requested ML instances......
2020-06-30 06:43:41 Starting - Preparing the instances for training...
2020-06-30 06:44:25 Downloading - Downloading input data...
2020-06-30 06:44:59 Training - Training image download completed. Training in progress.
2020-06-30 06:44:59 Uploading - Uploading generated training model
2020-06-30 06:44:59 Failed - Training job failed
2020-06-30 06:44:51,022 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-06-30 06:44:51,029 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-30 06:44:51,299 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-30 06:44:51,313 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-30 06:44:51,327 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-30 06:44:5

UnexpectedStatusException: Error for Training job tensorflow-training-2020-06-30-06-42-30-538: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/bin/python3 model.py --model_dir s3://sagemaker-us-west-2-378706508412/tensorflow-training-2020-06-30-06-42-30-538/model"

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')